## 1️⃣ Put skeleton data file names to an array named files_in_folder

In [1]:
import os
files_in_folder = []
data_path = "E:/my_courses/project/kinect dataset"
for file in os.listdir(data_path):
    if file.endswith("_screen.txt"):
        print(os.path.join(data_path, file))
        files_in_folder.append(file)

In [2]:
len(files_in_folder)

150

## 2️⃣ Get the number of frames in each file in files_in_folder and put it in frames_file array

In [3]:
path = "E:/my_courses/project/kinect dataset/"
frames_file = []
for i in files_in_folder:
    with open(path+i) as f:
        frames_file.append(len(f.readlines())/15)

### ✅ Calculate minimum frame number among files

In [4]:
min(frames_file)

42.0

### ✅ See number of frames per second some files have

In [6]:
5/121

0.04132231404958678

In [16]:
1/0.041

24.390243902439025

In [9]:
4/103

0.038834951456310676

In [15]:
1/0.038

26.315789473684212

In [11]:
4/96

0.041666666666666664

In [17]:
1/0.041

24.390243902439025

In [13]:
4/102

0.0392156862745098

In [14]:
1/0.039

25.641025641025642

In [18]:
4/101

0.039603960396039604

In [19]:
1/0.039

25.641025641025642

In [20]:
5/124

0.04032258064516129

In [21]:
1/0.04

25.0

In [43]:
4/107

0.037383177570093455

In [44]:
1/0.037

27.027027027027028

In [45]:
5/122

0.040983606557377046

In [46]:
1/0.04

25.0

In [48]:
0.5/0.04

12.5

### ✅ Finally 42 is good for windows size 

## 3️⃣ Move files to test and train folders

In [5]:
import shutil
test_counter = 0
path = "E:/my_courses/project/kinect dataset/"
path_to_copy = "E:/my_courses/project/final_files/dataset/"
for i in range(len(files_in_folder)):
    if i == test_counter:
        shutil.copy(path+files_in_folder[i], path_to_copy+"test")
        test_counter += 12
    else:
        shutil.copy(path+files_in_folder[i], path_to_copy+"train")


### ✅ 8 percent test data = 13

## 4️⃣ Remove some frames from each file (train or test) to reach 42 frames
##       Then move them to new folders. (train_2 or val)

In [2]:
import os
import numpy as np
path = "E:/my_courses/project/final_files/dataset/train/" #train  or test
path_save = "E:/my_courses/project/final_files/dataset/train_2/" #train_2  or val 

files_in_folder = []
for file in os.listdir(path):
    files_in_folder.append(file)
    
frames_file = []

for j in files_in_folder:
    with open(path+j) as f:
        
        for line in f.readlines():
            frames_file.append([int(float(x)) for x in (line.split())])  
            
        len_frames = int(len(frames_file)/15)
        print('len_1: ',len_frames)
        if len_frames >= 100: #remove first 25 and last 12 frames
            len_frames -= 37
            frames_file = frames_file[25*15:-12*15]
            print('len_frames: ',len_frames)
            
        num = int(len_frames/42)*42 
        part_forward = (num/(42-(len_frames - num)))
        rest = (len_frames - num)
        
        final_file = []
        

        counter = 15 
        print(j)
        i = 0 
        o = 0
        condition = False
        
        while True:
            round_i = round(i)
            
            final_file.append(frames_file[round_i*counter:(round_i+1)*counter])
            i += part_forward
                 
            o += 1
            if condition == True:
                break
                
            if condition == False and o == (42-(len_frames - num)):
                if  ((num) - (i-part_forward))<10:
                    break
                else:
                    condition = True

                
        if condition:  
            print('--')
            num += 1
        
        for s in range(num,len_frames):
            final_file.append(frames_file[s*counter:(s+1)*counter])    
        
        print('diff: ',(num) - (i-part_forward))
        print(o + len_frames-num)
        
        #save to file
        train_file = open(path_save+j, "w")
        for row in final_file:
            np.savetxt(train_file, row)
        train_file.close()
        frames_file = []
        print('---------------------end-------------------------')
        #save to file
        
        


### ✅ For whatching selected frames of specified file

In [20]:
path_save = "E:/my_courses/project/final_files/dataset/train_2/" 
import numpy as np 
import cv2
img = np.zeros((640,480))
with open(path_save+'a13_s01_e03_screen.txt') as f:
    
    c = 0
    k = 0
    print('yes')
    for line in f.readlines():
        
        seq_lines = ([int(float(x)) for x in (line.split())])
        img = cv2.circle(img,(seq_lines[0],seq_lines[1]),10,255,-1) 
        c += 1
        if c % 15 == 0:
            cv2.imwrite(path_save+'{}.jpg'.format(c),img)
            img = np.zeros((640,480))

yes


## 5️⃣ Convert skeleton data to image by keeping the label as name of image

### Normalization:
255*(x_i-min⁡(X))/(max⁡(X)-min⁡(X) )

In [54]:
# Head = 3  
# Neck = 2  
# Right_Shoulder = 8
# Right_Elbow = 9
# Right_Hand = 11
# Left_Shoulder = 4
# Left_Elbow = 5
# Left_Hand = 7
# Torso = 1
# Right_Hip = 16
# Right_Knee = 17
# Right_Foot = 19 
# Left_Hip = 12
# Left_Knee = 13
# Left_Foot = 15

In [26]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

train_2_path = "E:/my_courses/project/final_files/dataset/val/"#val,train_2,test
save_img_path = "E:/my_courses/project/final_files/dataset/val_dist_scale/"#train_dist_scale,val_dist_scale,test_dist_scale


for file in os.listdir(train_2_path):
    with open(train_2_path+file) as f:
        seq_lines = []

        label = file.split('.')[0][:-7]
        
        for line in f.readlines():
            seq_lines.append(([int(float(x)) for x in (line.split())]))
        
        seq_lines = np.array(seq_lines)
        X = seq_lines[:,0]
        Y = seq_lines[:,1]
        Z = seq_lines[:,2]
        
        X = X.reshape(X.reshape(15,-1).shape[1],15).transpose()
        X = X[8] - X # DISTANCE between torso and others
        X = np.delete(X, 1, 0) #delete torso 
        
        Z = Z.reshape(Z.reshape(15,-1).shape[1],15).transpose()
        div = np.delete(Z, 1, 0)
        
        X = X * (div/4500) # scale every points to 4.5 meter distance of kinect camera
        r = (255*(X-(X.min()))/((X.max())-(X.min()))).astype(int) # Normalization and put into to 0-255 range
        
        Y = Y.reshape(Y.reshape(15,-1).shape[1],15).transpose()
        Y = Y[8] - Y
        Y = np.delete(Y, 1, 0) #delete torso
        Y = Y * (div/4500)
        g = (255*(Y-(Y.min()))/((Y.max())-(Y.min()))).astype(int)
        
        Z = Z[8] - Z  
        Z = np.delete(Z, 1, 0) #delete torso
        b = (255*(Z-(Z.min()))/((Z.max())-(Z.min()))).astype(int)
        
        image = cv2.merge((r,g,b))
        cv2.imwrite(save_img_path+label+".jpg",image)        
         

    